In [12]:
import pandas as pd

In [78]:
forecast_file = 'result.csv'

forecast_df = pd.read_csv(forecast_file)

forecast_df['date'] = pd.to_datetime(forecast_df['data'])
forecast_df = forecast_df.drop(['data', 'Unnamed: 0'], axis=1).sort_values(by=['date'])

forecast_df.head()

,forecast,correct_forecast,host,date
0,0.0,0.0,all,2022-01-01 00:00:00
915840,0.0,0.0,cross,2022-01-01 00:00:00
596160,0.0,0.0,five,2022-01-01 00:00:00
1,0.0,0.0,all,2022-01-01 00:01:00
596161,0.0,0.0,five,2022-01-01 00:01:00


In [79]:
forecast_df_all = forecast_df[forecast_df['host']=='all']
forecast_df_cross = forecast_df[forecast_df['host']=='cross']
forecast_df_five = forecast_df[forecast_df['host']=='five']

df = forecast_df_all

d1 = '2022-08-04 13:32:00'
d2 = '2022-08-04 13:49:00'

forecast = df[(d1 <= df['date']) & (df['date'] <= d2)]['correct_forecast'].sum()
revenue = 1138046.30

print(d1, '-', d2)
print(f'\nПрогноз: {forecast:12.0f}\nВыручка: {revenue:12.0f}\nПотери:  \033[1m{forecast - revenue:12.0f}')

# for month in [10,11,7,8]:   
#     d1 = f'2022-{month if month > 9 else "0"+str(month)}-01 00:00:00'
#     d2 = f'2022-{month+1 if month > 8 else "0"+str(month+1)}-01 00:00:00'

#     print(f'{month:2.0f}:', d1, d2, round(df[(d1 <= df['date']) & (df['date'] < d2)]['forecast'].sum()))

2022-08-04 13:32:00 - 2022-08-04 13:49:00

Прогноз:      3081341
Выручка:      1138046
Потери:       1943295


In [80]:
lsr_file = 'C:/Users/User/Projects/LSR/lsr_with_revenue_202208100934.csv'

lsr_df = pd.read_csv(lsr_file)

lsr_df['start_datetime'] = pd.to_datetime(lsr_df['start_datetime'])
lsr_df['end_datetime'] = pd.to_datetime(lsr_df['end_datetime'])

lsr_df['revenue_all'] = lsr_df[['revenue_cross', 'revenue_five', 'revenue_okolo']].sum(axis='columns')

lsr_df = lsr_df[['issuenum', 'start_datetime', 'end_datetime', 'jira_losses', 'revenue_all', 'revenue_cross', 'revenue_five', 'revenue_okolo']]

lsr_df.head()

,issuenum,start_datetime,end_datetime,jira_losses,revenue_all,revenue_cross,revenue_five,revenue_okolo
0,243,2022-01-10 14:30:00,2022-01-10 15:15:00,892900.0,6154099.94,1610888.29,4187705.99,355505.66
1,281,2022-01-19 14:30:00,2022-01-19 15:15:00,3187813.0,5681393.37,1047743.96,4274132.48,359516.93
2,279,2022-01-23 17:36:00,2022-01-23 17:59:00,2453237.0,2866690.52,2634429.77,225757.91,6502.84
3,280,2022-01-24 12:08:00,2022-01-24 12:40:00,NaN,6715074.66,3030537.49,3468124.78,216412.39
4,295,2022-01-26 16:07:00,2022-01-26 18:27:00,NaN,31774209.31,15187800.18,15480542.37,1105866.76


In [82]:
forecast_all = []
forecast_cross = []
forecast_5ka = []

for i in range(len(lsr_df)):
    d1 = lsr_df.iloc[i]['start_datetime']
    d2 = lsr_df.iloc[i]['end_datetime']
    
#     if d1 > df['date'][0]:
    
    bigger_than_1h = (d2 - d1) > pd.Timedelta(1, "h")

    sum_df = forecast_df.loc[(d1 <= forecast_df['date']) & (forecast_df['date'] <= d2)].groupby(['host']).sum()
   
    if bigger_than_1h:
        forecast_type = 'forecast'
    else:
        forecast_type = 'correct_forecast'

    if len(sum_df) == 3:
        forecast_all.append(sum_df.loc['all', forecast_type])
        forecast_cross.append(sum_df.loc['cross', forecast_type])
        forecast_5ka.append(sum_df.loc['five', forecast_type])
    
    else:
        forecast_all.append(pd.NA)
        forecast_cross.append(pd.NA)
        forecast_5ka.append(pd.NA)
                            
lsr_df['forecast_all'] = forecast_all
lsr_df['forecast_cross'] = forecast_cross
lsr_df['forecast_five'] = forecast_5ka

lsr_df['losses_all'] = lsr_df['forecast_all'] - lsr_df['revenue_all']
lsr_df['losses_cross'] = lsr_df['forecast_cross'] - lsr_df['revenue_cross']
lsr_df['losses_five'] = lsr_df['forecast_five'] - lsr_df['revenue_five']

lsr_df.loc[lsr_df['losses_all'] < 0, 'losses_all'] = 0
lsr_df.loc[lsr_df['losses_cross'] < 0, 'losses_cross'] = 0
lsr_df.loc[lsr_df['losses_five'] < 0, 'losses_five'] = 0

lsr_df.to_excel('C:/Users/User/Projects/LSR/lsr_with_forecast.xlsx', index=False)

lsr_df 

,issuenum,start_datetime,end_datetime,jira_losses,revenue_all,revenue_cross,revenue_five,revenue_okolo,forecast_all,forecast_cross,forecast_five,losses_all,losses_cross,losses_five
0,243,2022-01-10 14:30:00,2022-01-10 15:15:00,892900.0,6154099.94,1610888.29,4187705.99,355505.66,6154099.94,1610888.29,4187705.99,0.0,0,0
1,281,2022-01-19 14:30:00,2022-01-19 15:15:00,3187813.0,5681393.37,1047743.96,4274132.48,359516.93,13590402.456842,6624508.150177,5147472.257367,7909009.086842,5576764.190177,873339.777367
2,279,2022-01-23 17:36:00,2022-01-23 17:59:00,2453237.0,2866690.52,2634429.77,225757.91,6502.84,6540799.671777,3229110.881635,2989910.620211,3674109.151777,594681.111635,2764152.710211
3,280,2022-01-24 12:08:00,2022-01-24 12:40:00,NaN,6715074.66,3030537.49,3468124.78,216412.39,9483373.84076,4262448.557176,4712294.927634,2768299.18076,1231911.067176,1244170.147634
4,295,2022-01-26 16:07:00,2022-01-26 18:27:00,NaN,31774209.31,15187800.18,15480542.37,1105866.76,37299717.158711,18386226.704871,17517632.541232,5525507.848711,3198426.524871,2037090.171232
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,370,2022-07-26 13:29:00,2022-07-26 13:38:00,588131.0,869711.15,539424.65,330286.50,NaN,1445136.910302,649698.747324,917376.007216,575425.760302,110274.097324,587089.507216
92,373,2022-07-27 15:01:00,2022-07-28 15:33:00,60000.0,97254441.68,45197057.21,52057384.47,NaN,107909446.557482,45302365.190589,51272847.913088,10655004.877482,105307.980589,0
93,371,2022-07-29 09:00:00,2022-07-29 10:05:00,NaN,8730845.82,4144000.76,4586845.06,NaN,8624196.090093,4057976.144946,4393741.077722,0,0,0
94,372,2022-07-29 14:25:00,2022-07-29 14:50:00,796951.0,2633043.35,1584255.31,1048788.04,NaN,3768751.561631,1753871.573352,1846757.310798,1135708.211631,169616.263352,797969.270798
